In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pointnet2.data.Indoor3DSemSegLoader import Indoor3DSemSeg
num_points = 4096
train_set = Indoor3DSemSeg(num_points, download=False)

In [3]:
# num of 3d models, num of points in each model, xyz cood
print(train_set.points.shape)

(16733, 4096, 3)


In [4]:
from matplotlib import pyplot
from mpl_toolkits.mplot3d import Axes3D
import random

fig = pyplot.figure()
ax = Axes3D(fig)

data_to_draw = train_set.points[12]

sequence_containing_x_vals = data_to_draw[:, 0]
sequence_containing_y_vals = data_to_draw[:, 1]
sequence_containing_z_vals = data_to_draw[:, 2]

ax.scatter(sequence_containing_x_vals, sequence_containing_y_vals, sequence_containing_z_vals)
pyplot.show()

<Figure size 640x480 with 1 Axes>

In [5]:
from torch.utils.data import DataLoader
batch_size = 1

train_loader = DataLoader(
    train_set,
    batch_size=batch_size,
    pin_memory=True,
    num_workers=8,
    shuffle=True,
)

In [6]:
from pointnet2.models.pointnet2_msg_sem import Pointnet2MSG
model = Pointnet2MSG(num_classes=13, input_channels=0, use_xyz=True)

In [7]:
import torch.nn as nn
from pointnet2.models.pointnet2_msg_sem import model_fn_decorator

model_fn = model_fn_decorator(nn.CrossEntropyLoss())

In [9]:
model.eval()
model.cuda()
for data in train_loader:
    _, loss, eval_res = model_fn(model, data, eval=True)
    break
features = model.get_features()

In [10]:
print(features.shape)

torch.Size([1, 128, 4096])


In [11]:
import torch
import numpy as np

B = 1
N = 2
inputs = torch.randn(B, N, 3).cuda()
labels = torch.from_numpy(np.random.randint(0, 3, size=B * N)).view(B, N).cuda()
model = Pointnet2MSG(num_classes=3, input_channels=0, use_xyz=True)

In [13]:
model.cuda()
_, _, _ = model_fn(model, (inputs, labels))
features = model.get_features()

In [14]:
print(features.shape)

torch.Size([1, 128, 2])
